In [1]:
using Distributions, Plots, LinearAlgebra, ForwardDiff, Optim
using DataFrames, CSV, Random

# Homework on MSLE

Recall that in the previous homework we have Model A and its log-likelihood function of observation $i$ is as follows.

\begin{align}
 y_i & = \alpha + x_i' \beta + v_i - u_i,\\
 v_i & \sim N(0, \sigma_v^2), \\
 u_i & \sim N^+(0, \sigma_u^2),
\end{align}
 
\begin{aligned}
L_i = - \ln \left(\frac{1}{2}\right) -\frac{1}{2}\ln (\sigma_v^2 + \sigma_u^2) + \ln
\phi\left(\frac{\epsilon_i}{\sqrt{\sigma_v^2 + \sigma_u^2}} \right) +
\ln \Phi\left(\frac{\mu_{*i}}{\sigma_*} \right),
\end{aligned}


where $\phi(z)$ and $\Phi(z)$ are the PDF and CDF of a standard normal distribution. Also,

\begin{aligned}
 \mu_{*i}  = \frac{-\sigma_u^2 \epsilon_i}{\sigma_v^2 + \sigma_u^2},\qquad
 \sigma_*^2  = \frac{\sigma_v^2  \sigma_u^2}{\sigma_v^2 + \sigma_u^2}. 
\end{aligned}


###### 1. Write a Julia function of the Model's log-likelihood function.
Write a function for Model A's log-likelihood that is suitable for estimating using the Monte Carlo simulation approach (MCSA of Section 1.4 in the lecture note). 

  - Name the function `NHN_msle`.
  - Use the Monte Carlo method (not the Quasi Monte Carlo method) to draw samples of $u_i^s$, $s=1,\ldots,S$.


###### 2. Empirical Estimation
The attached dataset, `sampledata.csv`, contains data of agricultural production from India. The variables are the follows. They have been converged to appropriate units (using log, etc.) so no further data processing is necessary.


|          |  |        |
|-------------------------------------|---|---------------------|
| yvar: rice output                   |   | Lland: land         |
| Plland: irrigated land              |   | Llabor: labor       |
| Lbull: bull cost                    |   | Lcost: other costs  |
| yr: production year                 |   | age: age of farmers |
| school: farmers' years of schooling |   | yr_1: same as year  |


Your work is to use `NHN_msle` to estimate the data with the following specification.

$yvar_i = \alpha + \beta_1 Lland_i + \beta_2 Plland_i + \beta_3 Llabor_i + \beta_4 Lbull_i + \beta_5 Lcost_i + \beta_6 yr_i + v_i - u_i$.
  
You may use the following code to read in the data.

```julia
####################
using DataFrames, CSV
df = DataFrame(CSV.File("sampledata.csv"))
y = df[:, "yvar"]       # the dep var
x = Matrix(df[:, 2:7])  # the indep vars, not including a constant
####################
```

***The required result is a table with three columns: the 1st column is the coefficients, the 2nd column is the standard errors, and the 3rd column is the $t$ statistics.***

  - The table is preferably in Dataframe. How to convert a matrix to a DataFrame? Ask ChatGPT!


###### 2.1: The Estimation Guidelines
  
  - You may use $-0.1$ as the initial values for all parameters. Or you may use the OLS result as the initial values for the $\alpha$ and $\beta$ coefficients (`x2=hcat(ones(size(y,1), 1), x); ols=inv(x2'x2)*(x2'y)`). Or, you may choose any initial values that seem to be reasonable choices. However, you _**cannot**_ use the true answer (provided below) as the initial values.

  - Set the value of $S$ (the number of random draws on $u$) to be $S=2^{10} -1$.

  - I strongly suggest that your program uses the `autodiff = :forward` option (which uses automatic differentiation) in the estimation.
  
    - The `autodiff = :forward` option puts stringent requirements on data types which may not easy to work out. I suggest that you start with your program without the option (which would then default to numerical finite differences that is easier on the data). After you have a working program, you may add the option back and see if it works. Most likely you'll have error messages and you have to work out the issues. If you can't make it work, that's fine. Try your best.  
  

- Hint 1: Look at $|g(x)|$ to judge whether it is converged. It should be smaller than the convergence criterion. 
  
- Hint 2: The answer from the MSLE should be close to but not exactly the same as that from the MLE.


In [2]:
df = DataFrame(CSV.File("C:\\Users\\User\\sampledata.csv"))
y = df[:, "yvar"]     
x = Matrix(df[:, 2:7]) 
x2=hcat(ones(size(y,1), 1), x)
ols=inv(x2'x2)*(x2'y)
println(" ")

In [3]:

function NHN_msle(y, x, α, β, log_σᵥ², log_σᵤ²)
    σᵥ² = exp(log_σᵥ²)
    σᵤ² = exp(log_σᵤ²)
    σᵥ = sqrt(σᵥ²)
    s = 2^10-1
    u_list=rand(Xoshiro(1234),truncated(Normal(0,σᵤ²),0,Inf),s)
    f(e, σᵥ) = pdf(Normal(0, σᵥ), e)     
    ϵ  = y .- α .- x*β
    logLike = Array{Real}(undef, size(y,1))
    for i in 1:size(y,1) 
        logLike[i] = log(sum(f.(ϵ[i,1] .+ u_list, σᵥ)/s))      
    end
    return -sum(logLike)
end

NHN_msle (generic function with 1 method)

In [4]:
##### start estimation #########
nofx = 6
init = [ols;[-0.1,-0.1]]
func1 = TwiceDifferentiable(vars -> NHN_msle(y, x, vars[1],
                            vars[2:nofx+1], vars[end-1], vars[end]),
                            ones(nofx+3) )

res1 = optimize(func1, init, Newton(), Optim.Options(g_tol = 1e-7, iterations = 500))
                              
coeff1 = Optim.minimizer(res1)        
hwk14_coeff = deepcopy(coeff1)                # keep _coevec untouched
hwk14_coeff[end-1:end] = exp.(hwk14_coeff[end-1:end])     # convert unit of the last two elements
_Hessian  = Optim.hessian!(func1, coeff1)  # Hessain evaluated at the coeff vector

var_cov_matrix = inv(_Hessian)   

stderror  = sqrt.(diag(var_cov_matrix))

stderror[end-1:end] = hwk14_coeff[end-1:end] .*stderror[end-1:end]  # convert the unit using the delta method
t_stats = hwk14_coeff ./ stderror
hwk14_table = hcat(hwk14_coeff, stderror, t_stats)

@show res1

println("The estimation table is")
hwk14_table |> display

res1 =  * Status: success

 * Candidate solution
    Final objective value:     9.434078e+01

 * Found with
    Algorithm:     Newton's Method

 * Convergence measures
    |x - x'|               = 2.07e-07 ≰ 0.0e+00
    |x - x'|/|x'|          = 5.91e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.41e-12 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 3.62e-14 ≰ 0.0e+00
    |g(x)|                 = 5.28e-08 ≤ 1.0e-07

 * Work counters
    Seconds run:   28  (vs limit Inf)
    Iterations:    11
    f(x) calls:    40
    ∇f(x) calls:   40
    ∇²f(x) calls:  11

The estimation table is


9×3 Matrix{Float64}:
  1.60073     0.350642     4.56515
  0.283302    0.0707285    4.00548
  0.216854    0.176079     1.23157
  1.16218     0.0803405   14.4657
 -0.427072    0.0596114   -7.16427
  0.00646413  0.0133768    0.483235
  0.0349059   0.00813086   4.29302
  0.0302617   0.0090278    3.35205
  0.5354      0.0400101   13.3816

In [5]:
noX=6
function NHN_mle(y, x, α, β, log_σᵥ², log_σᵤ²)
    σᵥ² = exp(log_σᵥ²)
    σᵤ² = exp(log_σᵤ²)
    ϵ  = y .- α .- x*β
    μ_star = (-σᵤ²*ϵ) / (σᵥ²+σᵤ²)

    llike = 0.0
    for i in eachindex(ϵ)
        σ_star = (σᵥ²*σᵤ²) / (σᵥ²+σᵤ²)
        llike += -log(0.5) - 0.5*log(σᵥ²+σᵤ²) + log(pdf(Normal(0,1), ϵ[i]/sqrt(σᵥ²+σᵤ²) )) + log(cdf(Normal(0,1), μ_star[i]/σ_star ))
    end
    return -llike
end
func = TwiceDifferentiable(vars -> NHN_mle(y, x, vars[1], vars[2:noX+1], vars[end-1], vars[end]),
                           ones(9); autodiff = :forward)

res = optimize(func, [ols; [-0.1,-0.1]], Newton(), Optim.Options(g_tol = 1e-9,iterations = 50) )    
_coevec = Optim.minimizer(res)
res_coeff = deepcopy(_coevec)                
res_coeff[end] = exp(_coevec[end])          
res_coeff[end-1] = exp(_coevec[end-1])  
 
res8_Hessian  = Optim.hessian!(func, _coevec)  

var_cov_matrix = inv(res8_Hessian)            
stderror  = sqrt.(diag(var_cov_matrix))
stderror[end] = res_coeff[end]*stderror[end]  
t_stats = res_coeff ./ stderror
res8_table = hcat(res_coeff, stderror, t_stats)

println("The estimation table is")
res8_table |> display

The estimation table is


9×3 Matrix{Float64}:
  1.59012    0.349898     4.54453
  0.286194   0.0698563    4.09689
  0.234652   0.17648      1.32963
  1.15546    0.0811715   14.2348
 -0.421433   0.0593928   -7.09569
  0.0066182  0.0130428    0.507422
  0.0340904  0.00803004   4.24536
  0.191584   0.162488     1.17907
  0.0974947  0.0593393    1.643

In [9]:
coeff = hcat(["msle";hwk14_coeff],["mle";res_coeff])
coeff |> display

10×2 Matrix{Any}:
   "msle"       "mle"
  1.60073      1.59012
  0.283302     0.286194
  0.216854     0.234652
  1.16218      1.15546
 -0.427072    -0.421433
  0.00646413   0.0066182
  0.0349059    0.0340904
  0.0302617    0.191584
  0.5354       0.0974947